# 도구(Tools)

많은 AI 애플리케이션은 자연어를 통해 사용자와 상호 작용합니다. 그러나 일부 사용 사례에서는 모델이 구조화된 입력을 사용하여 API, 데이터베이스 또는 파일 시스템과 같은 외부 시스템과 직접 인터페이스해야 합니다.

도구는 에이전트가 작업을 수행하기 위해 호출하는 구성 요소입니다. 

잘 정의된 입력과 출력을 통해 모델이 세상과 상호 작용할 수 있도록 하여 모델 기능을 확장합니다. 

도구는 호출 가능한 함수와 입력 스키마를 캡슐화합니다. 이들은 호환 가능한 채팅 모델에 전달될 수 있으며, 모델이 도구를 호출할지 여부와 어떤 인수로 호출할지를 결정할 수 있습니다.

## 사전 준비

환경 변수를 설정합니다.

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# 환경 변수 로드
load_dotenv(override=True)
# 추적을 위한 프로젝트 이름 설정
logging.langsmith("LangChain-V1-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-V1-Tutorial


## 도구 생성

### 기본 도구 정의

도구를 생성하는 가장 간단한 방법은 `@tool` 데코레이터를 사용하는 것입니다. 

기본적으로 함수의 docstring 이 모델이 도구를 언제 사용할지 이해하는 데 도움이 되는 도구의 설명이 됩니다.

즉, 정리하면 다음과 같습니다.

1. `@tool` 데코레이터를 사용하여 도구를 정의합니다.
2. 도구의 이름은 함수 이름에서 가져옵니다.
3. 도구의 설명은 함수의 docstring 에서 가져옵니다.
4. 도구의 입력 스키마는 함수의 매개변수와 타입 힌트에서 가져옵니다.

In [2]:
from langchain.tools import tool


@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"


# 도구 정보 확인
print(f"Tool name: {search_database.name}")
print(f"Tool description: {search_database.description}")

Tool name: search_database
Tool description: Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return


타입 힌트는 도구의 입력 스키마를 정의하므로 필수입니다. 

docstring은 모델이 도구의 목적을 이해하는 데 도움이 되도록 정보가 풍부하고 간결해야 합니다.

### 도구 속성 커스터마이징

#### 커스텀 도구 이름(Tool Name)

기본적으로 도구 이름은 함수 이름에서 가져옵니다. 더 설명적인 이름이 필요한 경우 이를 아래와 같이 재정의할 수 있습니다.

`@tool("커스텀 이름")` 을 사용하여 도구 이름을 재정의할 수 있습니다.

In [3]:
@tool("web_search")  # 커스텀 이름
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"


# 도구 이름 확인
print(f"Tool name: {search.name}")  # web_search

Tool name: web_search


#### 커스텀 도구 설명(Tool Description)

더 명확한 모델 가이드를 위해 자동 생성된 도구 설명을 재정의할 수 있습니다.

`@tool(description="커스텀 설명")` 을 사용하여 도구의 설명을 재정의할 수 있습니다.

In [4]:
@tool(
    "calculator",
    description="Performs arithmetic calculations. Use this for any math problems.",
)
def calc(expression: str) -> str:
    """Evaluate mathematical expressions."""
    return str(eval(expression))


# 도구 설명 확인
print(f"Tool description: {calc.description}")

Tool description: Performs arithmetic calculations. Use this for any math problems.


## pydantic 모델로 도구를 위한 스키마 정의

pydantic 모델을 활용하여 명확한 입력 스키마를 정의할 수 있습니다.

`@tool(args_schema=pydantic_model)` 을 사용하여 도구의 입력 스키마를 정의할 수 있습니다.

In [5]:
from pydantic import BaseModel, Field
from typing import Literal


class WeatherInput(BaseModel):
    """Input for weather queries."""

    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius", description="Temperature unit preference"
    )
    include_forecast: bool = Field(default=False, description="Include 5-day forecast")


@tool(args_schema=WeatherInput)
def get_weather(
    location: str, units: str = "celsius", include_forecast: bool = False
) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"현재 {location} 지역의 날씨는 {temp} {units[0].upper()} 도"
    if include_forecast:
        result += "\n다음 5일 날씨: 맑음"
    return result

In [6]:
# 도구 테스트
print(
    get_weather.invoke(
        {"location": "Seoul", "units": "celsius", "include_forecast": True}
    )
)

현재 Seoul 지역의 날씨는 22 C 도
다음 5일 날씨: 맑음


아래는 고의로 `units` 를 `celsiuss` 로 설정하여 오류를 발생시켜 보는 예제입니다.

아래의 경우 오류가 발생할 수 있는 이유는 입력 스키마에서 `Literal["celsius", "fahrenheit"]` 타입을 사용하였으나, 실제 입력값으로 `celsiuss` 를 입력했기 때문입니다.

In [7]:
# 도구 테스트
print(
    get_weather.invoke(
        {"location": "Seoul", "units": "celsiuss", "include_forecast": True}
    )
)

ValidationError: 1 validation error for WeatherInput
units
  Input should be 'celsius' or 'fahrenheit' [type=literal_error, input_value='celsiuss', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error

## 컨텍스트 접근

도구는 에이전트 상태, 런타임 컨텍스트 및 장기 메모리에 액세스할 수 있을 때 가장 강력합니다. 이를 통해 도구는 컨텍스트 인식 결정을 내리고, 응답을 개인화하며, 대화 전반에 걸쳐 정보를 유지할 수 있습니다.

도구는 `ToolRuntime` 매개변수를 통해 런타임 정보에 액세스할 수 있으며, 다음의 정보를 제공합니다.

- **State** - 실행을 통해 흐르는 변경 가능한 데이터 (메시지, 카운터, 커스텀 필드)
- **Context** - 사용자 ID, 세션 세부 정보 또는 애플리케이션별 구성과 같은 불변 구성
- **Store** - 대화 전반에 걸친 영구 장기 메모리
- **Stream Writer** - 도구가 실행될 때 커스텀 업데이트 스트리밍
- **Config** - 실행을 위한 RunnableConfig
- **Tool Call ID** - 현재 도구 호출의 ID

### ToolRuntime

`ToolRuntime`을 사용하여 단일 매개변수로 모든 런타임 정보에 액세스할 수 있습니다. 

도구 시그니처에 `runtime: ToolRuntime`을 추가하기만 하면 LLM에 노출되지 않고 자동으로 주입됩니다.

도구는 `ToolRuntime`을 사용하여 현재 그래프 상태에 액세스할 수 있습니다.

In [16]:
from typing import Literal, Optional, Dict, Any, List, Annotated, TypedDict
from pydantic import BaseModel
from langchain.tools import tool, ToolRuntime
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_teddynote.messages import stream_graph, invoke_graph
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage


# 현재 대화 상태 접근
@tool
def summarize_conversation(runtime: ToolRuntime) -> str:
    """Summarize the conversation so far."""
    # state 에서 메시지 접근
    messages = runtime.state.get("messages", [])
    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")
    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"


@tool
def get_user_preference(
    preference_name: Literal["food", "coding", "sports"],
    runtime: ToolRuntime,  # ToolRuntime 매개변수는 모델에 보이지 않습니다 (자동 주입)
) -> str:
    """Get a user preference value."""

    # context는 state에 저장하지 않고 별도의 context 객체로 inject됨
    preferences = {}
    if getattr(runtime, "context", None) is not None:
        # context dict 내 user_preferences
        preferences = runtime.context.user_preferences or {}
    return preferences.get(preference_name, "Have no information")


class CustomContext(BaseModel):
    user_preferences: Optional[Dict[str, Any]] = None


class CustomState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    user_preferences: Optional[Dict[str, Any]] = None


# 에이전트 생성
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(
    model,
    tools=[summarize_conversation, get_user_preference],
    system_prompt="You are a helpful assistant.",
    # checkpointer=InMemorySaver(),
    context_schema=CustomContext,
    # state_schema=CustomState,
)

config = {"configurable": {"thread_id": "3"}}
inputs = {"messages": [{"role": "user", "content": "내가 좋아하는 음식 알려줘"}]}

invoke_graph(
    agent,
    inputs=inputs,
    config=config,
    context=CustomContext(user_preferences={"food": "pizza"}),
)


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================
Tool Calls:
  get_user_preference (call_P2lUJ30JDa0ognrMVgsHabE4)
 Call ID: call_P2lUJ30JDa0ognrMVgsHabE4
  Args:
    preference_name: food

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================= Tool Message =================================
Name: get_user_preference

pizza


/Users/teddy/Dev/github/00-LangGraph-Tutorial/langgraph-v1-tutorial/.venv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=CustomContext(user_preferences={'food': 'pizza'}), input_type=CustomContext])
  return self.__pydantic_serializer__.to_python(



🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================

당신이 좋아하는 음식은 피자입니다. 다른 도움이 필요하신가요?


#### 상태 업데이트

`Command`를 사용하여 에이전트의 상태를 업데이트하거나 그래프의 실행 흐름을 제어할 수 있습니다.

In [ ]:
from dataclasses import dataclass
from langgraph.types import Command
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langchain.messages import AnyMessage, RemoveMessage, ToolMessage
from langchain_teddynote.messages import invoke_graph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated, List, Optional, Dict, Any
from pydantic import BaseModel, Field


class CustomContext(BaseModel):
    user_preferences: Optional[Dict[str, Any]] = None


class CustomState(BaseModel):
    user_name: str = Field(default="", description="The user's name")
    messages: Annotated[List[AnyMessage], add_messages]


# User Name 업데이트 도구
@tool
def update_user_name(new_name: str, runtime: ToolRuntime) -> Command:
    """Update the user's name."""
    return Command(
        update={
            "user_name": new_name,  # user_name 상태에 업데이트
            "messages": [
                ToolMessage(
                    content=f"Successfully updated user name to {new_name}",
                    tool_call_id=runtime.tool_call_id,  # runtime 에서 얻어온 tool_call_id 정보를 활용하여 업데이트
                )
            ],
        }
    )


@tool
def clear_messages(runtime: ToolRuntime) -> Command:
    """Clear all messages from the conversation history except the one whose tool_call_id matches the id we don't want to delete."""
    from langchain.messages import AIMessage

    messages = runtime.state.get("messages", [])

    to_remove_messages = []
    tool_call_id = runtime.tool_call_id

    for m in messages:
        if isinstance(m, AIMessage) and getattr(m, "tool_calls", None):
            # Tool Call ID 가 일치하지 않으면 삭제. Tool Call ID 가 일치하면 유지.
            if not any(call.get("id") == tool_call_id for call in m.tool_calls):
                to_remove_messages.append(m)
        else:
            to_remove_messages.append(m)

    removals = [RemoveMessage(id=m.id) for m in to_remove_messages]
    return Command(
        update={
            "messages": removals
            + [
                ToolMessage(
                    content=f"Successfully cleared all previous messages. Total of {len(removals)} deleted messages.",
                    tool_call_id=runtime.tool_call_id,
                )
            ]
        }
    )


# 에이전트 생성
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(
    model,
    tools=[update_user_name, clear_messages],
    system_prompt="You are a helpful assistant.",
    state_schema=CustomState,
    context_schema=CustomContext,  # 클래스 자체를 전달 (인스턴스가 아님)
    checkpointer=InMemorySaver(),
)

In [145]:
# 에이전트 실행
config = {"configurable": {"thread_id": "1"}}
invoke_graph(
    agent,
    inputs={"messages": [{"role": "user", "content": "내 이름은 테디야"}]},
    config=config,
)


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================
Tool Calls:
  update_user_name (call_8IpiHsQgS9fn0boyu48tEYce)
 Call ID: call_8IpiHsQgS9fn0boyu48tEYce
  Args:
    new_name: 테디

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
user_name:
테디
================================= Tool Message =================================
Name: update_user_name

Successfully updated user name to 테디

🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================

안녕하세요, 테디님! 어떻게 도와드릴까요?


In [146]:
invoke_graph(
    agent,
    inputs={"messages": [{"role": "user", "content": "내 이름은 사실 셜리야"}]},
    config=config,
)


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================
Tool Calls:
  update_user_name (call_hxxyUlNbwu4vUWn16886zUaU)
 Call ID: call_hxxyUlNbwu4vUWn16886zUaU
  Args:
    new_name: 셜리

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
user_name:
셜리
================================= Tool Message =================================
Name: update_user_name

Successfully updated user name to 셜리

🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================

이름을 셜리로 변경했어요. 셜리님, 무엇을 도와드릴까요?


In [147]:
messages = agent.get_state(config).values["messages"]
messages

[HumanMessage(content='내 이름은 테디야', additional_kwargs={}, response_metadata={}, id='fba17ade-3572-47b2-a939-70b10f8c7d19'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 87, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-Cb48yvwIjmuerToKynHuxqLnvXMiD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--ced8dc04-8ee6-486d-9fa2-6187f46dc220-0', tool_calls=[{'name': 'update_user_name', 'args': {'new_name': '테디'}, 'id': 'call_8IpiHsQgS9fn0boyu48tEYce', 'type': 'tool_call'}], usage_metadata={'input_tokens': 87, 'output_tokens': 17, 'total_tokens': 104, 'input_token_d

In [148]:
invoke_graph(
    agent,
    inputs={"messages": [{"role": "user", "content": "메시지 전부 삭제해줘"}]},
    config=config,
)


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================
Tool Calls:
  clear_messages (call_oYy9f618hea7imUfwxxIdxkz)
 Call ID: call_oYy9f618hea7imUfwxxIdxkz
  Args:

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


==========

In [149]:
invoke_graph(
    agent,
    inputs={"messages": [{"role": "user", "content": "내 이름이 사실 뭐라고 했지?"}]},
    config=config,
)


🔄 Node: model 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
================================== Ai Message ==================================

지금은 사용자의 이름을 알고 있지 않습니다. 원하시면 새 이름을 알려주시면 기억하도록 하겠습니다. 새로운 이름을 알려주시겠어요?


### 컨텍스트

`runtime.context`를 통해 사용자 ID, 세션 세부 정보 또는 애플리케이션별 구성과 같은 불변 구성 및 컨텍스트 데이터에 액세스할 수 있습니다.

In [150]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

# 사용자 데이터베이스 시뮬레이션
USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com",
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com",
    },
}


@dataclass
class UserContext:
    user_id: str


@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"


# 에이전트 생성
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(
    model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant.",
)

# 컨텍스트와 함께 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123"),
)

print(result["messages"][-1].content)

Your current balance is $5000. Is there anything else you would like to know or do?


### 메모리 (Store)

스토어를 사용하여 대화 전반에 걸쳐 영구 데이터에 액세스할 수 있습니다. 스토어는 `runtime.store`를 통해 액세스되며 사용자별 또는 애플리케이션별 데이터를 저장하고 검색할 수 있습니다.

In [ ]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


# 메모리 접근
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """Look up user info."""
    store = runtime.store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"


# 메모리 업데이트
@tool
def save_user_info(
    user_id: str, user_info: dict[str, Any], runtime: ToolRuntime
) -> str:
    """Save user info."""
    store = runtime.store
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info."


# 스토어와 에이전트 생성
store = InMemoryStore()
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(model, tools=[get_user_info, save_user_info], store=store)

# 첫 번째 세션: 사용자 정보 저장
print("=== Saving user info ===")
result1 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev",
            }
        ]
    }
)
print(result1["messages"][-1].content)

# 두 번째 세션: 사용자 정보 가져오기
print("\n=== Getting user info ===")
result2 = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Get user info for user with id 'abc123'"}
        ]
    }
)
print(result2["messages"][-1].content)

### Stream Writer

`runtime.stream_writer`를 사용하여 도구가 실행될 때 커스텀 업데이트를 스트리밍할 수 있습니다. 이는 도구가 수행하는 작업에 대해 사용자에게 실시간 피드백을 제공하는 데 유용합니다.

In [ ]:
from langchain.tools import tool, ToolRuntime


@tool
def get_weather_with_updates(city: str, runtime: ToolRuntime) -> str:
    """Get weather for a given city."""
    writer = runtime.stream_writer

    # 도구가 실행될 때 커스텀 업데이트 스트리밍
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")

    return f"It's always sunny in {city}!"


# 참고: runtime.stream_writer를 도구 내에서 사용하는 경우,
# 도구는 LangGraph 실행 컨텍스트 내에서 호출되어야 합니다.

In [155]:
from langchain_teddynote.messages import stream_graph

# 스토어와 에이전트 생성
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(model, tools=[get_weather_with_updates])

inputs = {"messages": [{"role": "user", "content": "서울 날씨 알려줘"}]}

for chunk in agent.stream(inputs, stream_mode="custom"):
    print(chunk)

# stream_graph(agent, inputs=inputs)

Looking up data for city: 서울
Acquired data for city: 서울
